# Kikao cha 1 – Kuanza Mazungumzo (Foundry Local)

Notebook hii inaanzisha Foundry Local, inapakua alias ya mfano uliopendekezwa, na inafanya mazungumzo ya kawaida na ya mtiririko.


# Hali
Kikao hiki kinaanzisha kiwango cha chini kabisa cha kupata modeli ndogo ya lugha ya ndani ikijibu kupitia Foundry Local. Utakuwa:
- Kusakinisha SDK / utegemezi wa mteja.
- Kuanza meneja wa Foundry Local kwa jina la utani lililochaguliwa (chaguo-msingi: `phi-4-mini`).
- Kutumia kiraka cha kujihami ili kuvumilia sehemu za hiari katika metadata ya modeli.
- Kutuma ombi la kawaida la kukamilisha mazungumzo.
- Kutiririsha majibu tokeni kwa tokeni.

Lengo ni kuthibitisha muda wako wa ndani & njia ya mtandao kabla ya kuendelea na RAG, uelekezaji, au mawakala.


### Maelezo: Usakinishaji wa Vitegemezi
Inasakinisha vifurushi vya Python vinavyohitajika kwa mtiririko huu mdogo wa mazungumzo:
- `foundry-local-sdk`: Kusimamia mifano ya ndani na mzunguko wa maisha ya huduma.
- `openai`: Muundo wa mteja unaofahamika kwa kukamilisha mazungumzo.
- `rich`: Uchapishaji mzuri kwa matokeo ya wazi kwenye daftari.

Kuendesha tena ni salama (idempotent). Ruka ikiwa mazingira yako tayari yana hivi.


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Maelezo: Uingizaji Muhimu  
Inaleta moduli zinazotumika katika daftari:  
- `FoundryLocalManager` kuwasiliana na mazingira ya modeli ya ndani.  
- Mteja wa `OpenAI` ili tuweze kutumia tena uso wa API wa mazungumzo unaojulikana.  
- `rich.print` kwa matokeo yenye mtindo.  

Hakuna miito ya mtandao inayotokea hapa—hii inaandaa tu nafasi ya majina.


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### Ufafanuzi: Uanzishaji wa Meneja & Marekebisho ya Metadata
Inaanzisha `FoundryLocalManager` kwa jina la utani lililochaguliwa na kutekeleza marekebisho ya kinga ili kushughulikia majibu ya huduma ambapo `promptTemplate` inaweza kuwa `null`.

Matokeo muhimu:
- Inathibitisha hali ya huduma na endpoint.
- Inaorodhesha mifano iliyohifadhiwa (inathibitisha hifadhi ya ndani).
- Inatatua kitambulisho halisi cha mfano kwa jina la utani (kinachotumika katika simu za mazungumzo baadaye).

Ikiwa utakutana na masuala ya uthibitishaji katika metadata ya huduma ghafi, muundo huu unaonyesha jinsi ya kusafisha bila kubadilisha SDK.


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### Maelezo: Kukamilisha Mazungumzo ya Msingi
Inaunda mteja anayelingana na `OpenAI` akielekeza kwenye mwisho wa ndani wa Foundry na kutekeleza kukamilisha moja ya mazungumzo yasiyo ya mtiririko. Lengo hapa:
- Hakikisha mfano unajibu bila hitilafu.
- Thibitisha muda wa majibu / muundo wa matokeo.
- Weka `max_tokens` kuwa ya wastani ili kuhifadhi rasilimali.

Ikiwa hii itashindwa, hakikisha tena kwamba huduma ya Foundry Local inaendelea na alias inatatuliwa kwa usahihi.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### Maelezo: Kukamilisha Gumzo kwa Kutiririsha
Inaonyesha utiririshaji wa tokeni kwa kuboresha ucheleweshaji unaoonekana na UX ya maingiliano. Kitanzi kinaonyesha mabadiliko ya hatua kwa hatua yanapowasili:
- Inafaa kwa UI za gumzo ambapo matokeo ya awali ya sehemu ni muhimu.
- Inakuruhusu kupima kasi ya tokeni dhidi ya ucheleweshaji wa kukamilisha kwa jumla.

Unaweza kubadilisha muundo huu kukusanya tokeni, kusasisha wijeti ya maendeleo, au kusitisha kizazi katikati.


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kwa usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya kiasili inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
